In [18]:
# Read csv, 
# remove unuseful columns
# split every row in 2
import pandas as pd



In [19]:
dataset = pd.read_csv("dockless.csv")

In [20]:
dataset.head(5)

,TripID,StartDate,StartTime,EndDate,EndTime,TripDuration,TripDistance,StartLatitude,StartLongitude,EndLatitude,EndLongitude,DayOfWeek,HourNum
0,0000045c-2677-3a7d-4b73-cad99a57,2019-06-26,19:30,2019-06-26,19:30,3.0,0.000,38.253,-85.756,38.253,-85.755,4,19
1,00008c1a-899b-8596-970f-9f6bf495,2019-07-03,11:00,2019-07-03,11:15,6.0,0.640,38.217,-85.757,38.221,-85.763,4,11
2,000096c8-bac3-4d6f-7ebb-30b957ee,2019-05-04,21:15,2019-05-04,21:30,7.0,0.684,38.221,-85.762,38.223,-85.764,7,21
3,0000f36b-a0c7-0378-3f34-e0fb80f6,2019-05-07,17:30,2019-05-07,18:00,32.0,1.740,38.254,-85.717,38.254,-85.707,3,17
4,00011cdb-4801-1ce5-a15d-6b9a539a,2019-06-09,13:30,2019-06-09,13:45,10.0,0.808,38.251,-85.758,38.257,-85.760,1,13


In [21]:
# Drop column and split dataset in 2 for pick-ups and returns
dataset_start = dataset.drop(["TripDistance"], axis=1)
dataset_end = dataset_start.copy()

In [22]:
dataset_start = dataset_start.drop(["EndDate", "EndTime", "EndLatitude", "EndLongitude"], axis=1)
dataset_start = dataset_start.rename(columns={"TripID": "trip_id", "StartDate": "date", "StartTime": "time", "StartLatitude": "lat", "StartLongitude": "long", "DayOfWeek": "week_day", "TripDuration": "duration", "HourNum": "hour"})
dataset_end = dataset_end.drop(["StartDate", "StartTime", "StartLatitude", "StartLongitude"], axis=1)
dataset_end = dataset_end.rename(columns={"TripID": "trip_id", "EndDate": "date", "EndTime": "time", "EndLatitude": "lat", "EndLongitude": "long", "DayOfWeek": "week_day", "TripDuration": "duration", "HourNum": "hour"})

print("Datasets have same shape?")
print(dataset_end.shape, dataset_start.shape)


Datasets have same shape?
(300550, 8) (300550, 8)


In [23]:
# Add column to specify if pickup or return: type: (if pickup, if return)
dataset_start["type"] = "pickup"
dataset_end["type"] = "return"

In [24]:
dataset_start.head()

,trip_id,date,time,duration,lat,long,week_day,hour,type
0,0000045c-2677-3a7d-4b73-cad99a57,2019-06-26,19:30,3.0,38.253,-85.756,4,19,pickup
1,00008c1a-899b-8596-970f-9f6bf495,2019-07-03,11:00,6.0,38.217,-85.757,4,11,pickup
2,000096c8-bac3-4d6f-7ebb-30b957ee,2019-05-04,21:15,7.0,38.221,-85.762,7,21,pickup
3,0000f36b-a0c7-0378-3f34-e0fb80f6,2019-05-07,17:30,32.0,38.254,-85.717,3,17,pickup
4,00011cdb-4801-1ce5-a15d-6b9a539a,2019-06-09,13:30,10.0,38.251,-85.758,1,13,pickup


In [25]:
dataset_end.head()

,trip_id,date,time,duration,lat,long,week_day,hour,type
0,0000045c-2677-3a7d-4b73-cad99a57,2019-06-26,19:30,3.0,38.253,-85.755,4,19,return
1,00008c1a-899b-8596-970f-9f6bf495,2019-07-03,11:15,6.0,38.221,-85.763,4,11,return
2,000096c8-bac3-4d6f-7ebb-30b957ee,2019-05-04,21:30,7.0,38.223,-85.764,7,21,return
3,0000f36b-a0c7-0378-3f34-e0fb80f6,2019-05-07,18:00,32.0,38.254,-85.707,3,17,return
4,00011cdb-4801-1ce5-a15d-6b9a539a,2019-06-09,13:45,10.0,38.257,-85.760,1,13,return


In [26]:
# Concatenate and sort by date_time
dataset = pd.concat([dataset_start, dataset_end]).sort_values(["date", "time"])

#Add numeric incremental ids for trips
dataset['trip_id'] = pd.factorize(dataset.trip_id)[0]

#Round columns floats 
dataset = dataset.round({'lat': 3, 'long': 3})

In [27]:
dataset.shape

(601100, 9)

In [28]:
dataset.head()

,trip_id,date,time,duration,lat,long,week_day,hour,type
183031,0,2018-08-09,10:30,30.0,38.253,-85.741,5,10,pickup
187900,1,2018-08-09,10:30,2.0,38.251,-85.757,5,10,pickup
190415,2,2018-08-09,10:30,19.0,38.253,-85.739,5,10,pickup
196463,3,2018-08-09,10:30,9.0,38.251,-85.758,5,10,pickup
187900,1,2018-08-09,10:30,2.0,38.251,-85.758,5,10,return


In [29]:
dataset.to_csv("../producer/dockless-vehicles.csv", index=False)